In [72]:
import urllib
import urllib.request
import json
import pandas as pd

url = 'https://ussouthcentral.services.azureml.net/workspaces/5e279eff6a844bbe857435d4896e4b78/services/3d48b2c8f66245069a596a602cb7e500/execute?api-version=2.0&details=true'
api_key = 'GgoR7dMdgFe7bNVHGjEMa99QdchTI8yRcZHVwtj3QR4hqBaP2FZN7LyzMYLkLRwJ51CqBbpZaCVJLS41kmJsfA==' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

productIds = pd.read_csv("product_ids.csv")
reviews = pd.read_csv("reviews.csv")

reviews["sentiment"] = "I have not changed"

# iterate over rows with iterrows()
for index, row in reviews.iterrows():
    data =  {

        "Inputs": {

                "input1":
                {
                    "ColumnNames": ["Col2"],
                    "Values": [ [ row['reviews.text'] ] ]
                },        },
            "GlobalParameters": {
        }
    }
    #Encode our data input into json
    body = str.encode(json.dumps(data))
    #Make API call to provided url
    req = urllib.request.Request(url, body, headers)
    #Open our HTTP object
    response = urllib.request.urlopen(req)
    # Convert HTTP object from bytes to text
    resp_text = urllib.request.urlopen(req).read().decode('UTF-8')
    # Use loads to decode from text
    json_obj = json.loads(resp_text)
    #Parse Object and assign to sentiment column
    reviews.at[index,'sentiment'] = json_obj['Results']['output1']['value']['Values'][0][1]
    #Print sentiment for that review
    print(reviews.at[index, 'sentiment'])
    

0.233655631542206
0.237073600292206
0.209400147199631
0.168145537376404
0.236995294690132
0.272302001714706
0.17985163629055
0.184177190065384
0.168206110596657
0.189110532402992
0.184177190065384
0.200342133641243
0.206835091114044
0.168206110596657
0.171707585453987
0.237073600292206
0.176405131816864
0.178352057933807
0.209400147199631
0.236995294690132
0.168206110596657
0.356823951005936
0.263864129781723
0.13160540163517
0.236484959721565
0.173667296767235
0.249684497714043
0.200359761714935
0.250295221805573
0.146968066692352
0.178227663040161
0.161372274160385
0.202407985925674
0.18778096139431
0.133715659379959
0.294178783893585
0.209130644798279
0.168145537376404
0.208295553922653
0.253202736377716
0.126839131116867
0.209400147199631
0.234854832291603
0.258878856897354
0.166129097342491
0.302797645330429
0.128867536783218
0.120600201189518
0.296785265207291
0.184330850839615
0.155101388692856
0.186866089701653
0.151176795363426
0.229723259806633
0.171325594186783
0.14172090590

In [73]:

#Recasts our object as a float datatype
reviews['sentiment'] = reviews['sentiment'].astype(float)
#Aggregates the Sentiment grouped by asin number
sentimentbyAsins = reviews.groupby('asins').agg({'sentiment' : ['max', 'min', 'mean']}).reset_index()
#Renames the columns of our resulting dataframe
sentimentbyAsins.columns = ['asins', 'max', 'min', 'mean']
#Assigns sentiment score to productsId dataset joining on asins
merged_inner = pd.merge(left=productIds,right=sentimentbyAsins, left_on='asins', right_on='asins')
#Compare the two dataframe
print(sentimentbyAsins)
print(merged_inner)


         asins       max       min      mean
0   B00IOY8XWQ  0.519931  0.084384  0.213655
1   B00IOYAM4I  0.300774  0.095642  0.221019
2   B00QFQRELG  0.280988  0.159689  0.224002
3   B00REQKWGA  0.387073  0.104161  0.233748
4   B00VINDBJK  0.262137  0.136910  0.181753
5   B00ZV9PXP2  0.245874  0.174946  0.209246
6   B010CEHQTG  0.265862  0.146695  0.216177
7   B017JG41PC  0.325247  0.108968  0.224688
8   B0189XYY0Q  0.260884  0.152276  0.213728
9   B018Y224PY  0.237074  0.156134  0.178688
10  B018Y225IA  0.275729  0.128878  0.185717
11  B018Y22BI4  0.237074  0.209019  0.219215
12  B018Y22C2Y  0.242369  0.141945  0.186784
13  B018Y23MNM  0.265988  0.150477  0.207724
14  B01ACEKAJY  0.314428  0.073090  0.205741
15  B01AHB9C1E  0.409144  0.163250  0.235217
16  B01AHB9CN2  0.270847  0.094069  0.206815
17  B01AHB9CYG  0.923320  0.073090  0.226982
18  B01AHBBG04  0.309830  0.077786  0.167097
19  B01AHBDCKQ  0.297079  0.132778  0.203285
20  B01BH83OOM  0.353324  0.128047  0.199096
21  B01J24